In [2]:
import pandas as pd
import numpy as np

In [3]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [4]:
data = pd.read_csv("Data/пп_станции.xlsx - Лист1.csv")
data.head()

,Станция,Номер линии,Дата,4/3/2024,4/2/2024,4/1/2024,3/31/2024,3/30/2024,3/29/2024,3/28/2024,3/27/2024,3/26/2024,3/25/2024,3/24/2024,3/23/2024,3/22/2024,3/21/2024,3/20/2024,3/19/2024,3/18/2024,3/17/2024,3/16/2024,3/15/2024,3/14/2024,3/13/2024,...,1/25/2024,1/24/2024,1/23/2024,1/22/2024,1/21/2024,1/20/2024,1/19/2024,1/18/2024,1/17/2024,1/16/2024,1/15/2024,1/14/2024,1/13/2024,1/12/2024,1/11/2024,1/10/2024,1/9/2024,1/8/2024,1/7/2024,1/6/2024,1/5/2024,1/4/2024,1/3/2024,1/2/2024,1/1/2024
0,Б.Рокоссовского,8,СОКОЛЬНИЧЕСКАЯ,16671,15113,2905,13998,12742,1217,17624,12097,14831,7792,9609,1229,2135,6895,1041,18651,4943,5619,17125,19679,8482,3261,...,2600,14185,7410,18924,10265,3061,1510,10798,19898,7645,17944,16665,14355,10499,18189,7280,4669,3812,5816,15740,13714,2561,11209,2722,4835
1,Черкизовская,8,СОКОЛЬНИЧЕСКАЯ,6248,5441,7991,14864,10015,9721,6623,13030,12372,17814,15379,2339,14576,18593,2588,6929,7277,3129,17656,1294,4426,4148,...,2485,13333,11854,1667,5797,12029,1828,6468,12877,7645,15416,17947,9090,3849,17700,13486,5061,3663,11353,6924,6496,15991,7029,3999,15638
2,Преображенск. пл,8,СОКОЛЬНИЧЕСКАЯ,14094,1132,6556,7265,4001,10961,6852,13961,11940,2349,19378,14975,12584,3391,12367,7694,5643,14169,6267,7095,2167,7788,...,7813,7735,10134,14017,8169,10657,12490,18196,5698,18122,10740,12682,9135,19574,4361,8176,9745,13281,15090,13309,8365,2811,5004,17068,17117
3,Сокольники СЛ,8,СОКОЛЬНИЧЕСКАЯ,18651,10083,6488,6409,9446,12051,15041,6793,3010,19196,17958,13304,19168,16050,8283,1348,8076,9769,13495,13672,16816,18654,...,3279,15963,10741,6518,2690,16002,15438,18937,11864,19236,7969,12214,8669,7326,7890,3637,14001,12049,15972,15545,18759,5573,15017,14176,5465
4,Красносельская,8,СОКОЛЬНИЧЕСКАЯ,7391,19339,15424,10725,5965,14682,18134,2055,9929,9980,10764,5599,9113,4628,11462,18312,18938,3711,10294,13716,8803,1803,...,13782,18008,14760,2836,1210,8202,17022,11939,7065,8814,7338,7490,3529,17303,8012,14194,7615,9321,16152,4581,7746,8936,11373,19561,1727


In [5]:
unique_stations = data["Станция"].unique()
unique_stations_df = pd.DataFrame(unique_stations, columns=["Станция"])
unique_stations_df.to_csv("Data/unique_stations.csv", index=False)

In [6]:
data["Станция"].value_counts()

Станция
Авиамоторная БКЛ    2
Минская             2
Раменки             2
Фонвизинская        2
Бутырская           2
                   ..
Новоясеневская      1
Ясенево             1
Тёплый стан         1
Коньково            1
Текстильщики СЦ     1
Name: count, Length: 314, dtype: int64

In [7]:
station_counts = data["Станция"].value_counts()
repeated_stations = station_counts[station_counts >= 2].index.tolist()
data[data["Станция"].isin(repeated_stations)]

,Станция,Номер линии,Дата,4/3/2024,4/2/2024,4/1/2024,3/31/2024,3/30/2024,3/29/2024,3/28/2024,3/27/2024,3/26/2024,3/25/2024,3/24/2024,3/23/2024,3/22/2024,3/21/2024,3/20/2024,3/19/2024,3/18/2024,3/17/2024,3/16/2024,3/15/2024,3/14/2024,3/13/2024,...,1/25/2024,1/24/2024,1/23/2024,1/22/2024,1/21/2024,1/20/2024,1/19/2024,1/18/2024,1/17/2024,1/16/2024,1/15/2024,1/14/2024,1/13/2024,1/12/2024,1/11/2024,1/10/2024,1/9/2024,1/8/2024,1/7/2024,1/6/2024,1/5/2024,1/4/2024,1/3/2024,1/2/2024,1/1/2024
37,Каширская,9,ЗАМОСКВОРЕЦКАЯ,5708,15683,5308,14251,8726,4668,13089,6668,7336,4571,13515,15999,2613,3301,6984,2869,10760,16074,18400,16802,9846,10085,...,4942,17835,2073,8599,2882,3799,3354,3898,16402,18497,13920,13503,17378,10338,2503,17782,1392,15969,13032,9069,14267,1585,18104,4951,12810
38,Каховская,11,КАХОВСКАЯ,6072,19477,16945,11455,2136,12473,16279,11631,6410,6950,17831,16595,14331,11385,2851,14449,16851,2721,8072,8337,4517,11879,...,12508,6133,1097,19357,5237,19155,17153,12428,8343,16810,13322,7251,15698,11350,13837,2766,16380,16798,8548,18717,16900,18281,8372,6024,16469
39,Варшавская,11,КАХОВСКАЯ,19342,12788,14576,17024,13852,8192,18656,18617,9452,7968,10094,7878,4646,17649,1998,15191,7985,5263,17800,5910,4035,5101,...,5374,9689,4053,11943,4903,4894,16529,15796,7747,16925,5335,18747,16510,7571,4021,8645,17993,1812,17853,6861,4455,14334,11472,6434,5815
231,Бутырская,14,КАЛИНИНСКАЯ,4877,18797,6819,14730,7402,8297,11777,17132,11362,8736,11891,15443,17274,1792,5128,3530,7938,1178,1834,2059,9229,12469,...,6717,18728,2091,10799,19777,2276,4163,2058,12037,3962,6923,6633,1750,19095,19377,9554,1968,9798,19559,12905,17249,12461,17781,13271,13056
232,Фонвизинская,14,КАЛИНИНСКАЯ,4661,18964,2452,4643,10070,8156,2915,13319,1761,17969,2560,10044,3129,1732,18395,16865,2582,6387,15051,4265,6318,15617,...,1043,13715,8548,12833,13804,13781,17832,7710,19261,13243,3345,8281,5522,14236,9464,15699,19429,16906,14248,18820,5859,8126,1043,6521,15362
233,Бутырская,18,ЛЮБЛИНСКАЯ,14862,18424,10024,8392,1292,8667,13308,13846,18912,9533,17119,2918,11358,1457,2615,1087,17826,3694,14929,6947,11194,2899,...,2868,7222,16646,17832,6990,15624,7668,12198,16940,5450,4979,6552,3089,19104,14885,3661,9042,14411,17355,12705,9056,17021,19488,11337,10689
234,Фонвизинская,18,ЛЮБЛИНСКАЯ,13709,7346,1510,10713,5735,1698,16743,1760,5510,19988,2489,16364,12920,10810,1540,14567,4787,5714,5001,19495,2133,13491,...,10902,15544,15317,15250,14082,9922,13896,7669,19254,2196,8980,16639,1378,12925,18704,5649,10915,1155,18398,12228,5334,18011,11037,9832,10543
236,Минская,14,КАЛИНИНСКАЯ,9539,2487,7753,6482,18688,18011,9623,5159,14618,12758,5177,1792,6777,17681,11843,15115,14109,13024,18297,1719,16749,13231,...,19494,19336,18206,12335,8030,15065,13871,11760,12423,18923,7411,19478,6102,4272,12192,3029,5811,11707,16777,5862,5575,12756,13441,6174,18486
238,Раменки,14,КАЛИНИНСКАЯ,13995,11683,8990,2696,7779,10842,18188,10429,2369,2105,11288,6274,1630,2660,11367,16710,14647,8748,1029,9634,11490,3988,...,12836,13320,18841,10889,3630,6715,15644,3310,19589,19491,4725,16103,2742,12344,13398,6334,12317,3003,4023,14961,2370,17956,6036,2340,12752
240,Минская,1691,СОЛНЦЕВСКАЯ ЛИН.,18069,4486,17365,12172,19582,4274,17830,6153,9909,15638,4686,8082,16928,18129,7431,16179,17207,7777,1584,11713,19658,7096,...,13147,7208,10171,3790,8599,12162,6353,12279,19317,2171,6787,2357,16851,7865,12389,14020,1919,7388,2470,18762,9836,12075,16998,5291,9876


In [8]:
station_counts[station_counts >= 2]

Станция
Авиамоторная БКЛ    2
Минская             2
Раменки             2
Фонвизинская        2
Бутырская           2
Варшавская          2
Каховская           2
Каширская           2
Name: count, dtype: int64

In [9]:
data.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,...,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321
Станция,Б.Рокоссовского,Черкизовская,Преображенск. пл,Сокольники СЛ,Красносельская,Комсомольск. СЛ,Красные ворота,Чистые пруды,Лубянка,Охотный ряд,Кропоткинская,Парк культуры СЛ,Фрунзенская,Спортивная,Воробьёвы горы,Университет,Пр-т Вернадск.СЛ,Юго-западная,Красногвардейск.,Домодедовская,Орехово,Царицыно,Кантемировская,Коломенская,Автозаводская,...,Пр-т Вернад. БКЛ,Мичурин.пр-т БКЛ,Аминьевская,Давыдково,Кунцевская БКЛ,Терехово,Марьина Роща БКЛ,Рижская БКЛ,Сокольники БКЛ,Текстильщики БКЛ,Печатники БКЛ,Нагатинский З-н,Кленовый бульвар,Варшавская,Каширская,Нижегород-я БКЛ,Каширская (Зам),Нижегород-я НБС,К,Пыхтино,Аэропорт Внуково,Яхромская,Лианозово,Физтех,Текстильщики СЦ
Номер линии,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,9,9,9,9,9,9,9,...,1749,1749,1749,1749,1749,1749,1749,1749,1749,1749,1749,1749,1749,1749,1749,1749,9,1779,1749,1691,1691,18,18,18,1749
Дата,СОКОЛЬНИЧЕСКАЯ,СОКОЛЬНИЧЕСКАЯ,СОКОЛЬНИЧЕСКАЯ,СОКОЛЬНИЧЕСКАЯ,СОКОЛЬНИЧЕСКАЯ,СОКОЛЬНИЧЕСКАЯ,СОКОЛЬНИЧЕСКАЯ,СОКОЛЬНИЧЕСКАЯ,СОКОЛЬНИЧЕСКАЯ,СОКОЛЬНИЧЕСКАЯ,СОКОЛЬНИЧЕСКАЯ,СОКОЛЬНИЧЕСКАЯ,СОКОЛЬНИЧЕСКАЯ,СОКОЛЬНИЧЕСКАЯ,СОКОЛЬНИЧЕСКАЯ,СОКОЛЬНИЧЕСКАЯ,СОКОЛЬНИЧЕСКАЯ,СОКОЛЬНИЧЕСКАЯ,ЗАМОСКВОРЕЦКАЯ,ЗАМОСКВОРЕЦКАЯ,ЗАМОСКВОРЕЦКАЯ,ЗАМОСКВОРЕЦКАЯ,ЗАМОСКВОРЕЦКАЯ,ЗАМОСКВОРЕЦКАЯ,ЗАМОСКВОРЕЦКАЯ,...,БКЛ,БКЛ,БКЛ,БКЛ,БКЛ,БКЛ,БКЛ,БКЛ,БКЛ,БКЛ,БКЛ,БКЛ,БКЛ,БКЛ,БКЛ,БКЛ,ЗАМОСКВОРЕЦКАЯ,НЕКРАСОВСКАЯ,БКЛ,СОЛНЦЕВСКАЯ ЛИН.,СОЛНЦЕВСКАЯ ЛИН.,ЛЮБЛИНСКАЯ,ЛЮБЛИНСКАЯ,ЛЮБЛИНСКАЯ,БКЛ
4/3/2024,16671,6248,14094,18651,7391,8883,3067,12873,8554,19856,13207,9390,8103,10651,18457,1015,4512,12444,3325,12810,15135,2211,13849,6937,15508,...,9972,10055,14672,6784,8396,8675,6316,7658,17577,2595,9718,7478,1641,15986,15460,3362,18534,16779,18248,8390,15787,9813,18978,1476,18055
4/2/2024,15113,5441,1132,10083,19339,15442,7139,11038,15482,7671,5076,12391,5998,5511,19841,18249,11774,9441,9598,11461,3636,10845,10743,8057,8627,...,2208,18288,2210,8509,13152,13327,19000,1113,2735,8133,15531,10771,5191,15409,15384,1525,5763,14810,8141,16045,10257,19663,8499,2893,18422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1/5/2024,13714,6496,8365,18759,7746,5709,1873,15867,4428,13439,7559,8084,16654,19416,17335,14184,9052,1511,2123,3786,8454,9311,3683,2127,11520,...,9002,18205,18901,5584,13838,5400,13146,11936,6687,1665,7309,3137,6204,4705,10024,7862,15306,15953,3694,14040,19801,14759,5616,3834,13438
1/4/2024,2561,15991,2811,5573,8936,15712,2772,11590,10987,13708,18383,13101,5394,1975,3553,2853,4839,3300,13437,11727,7047,8101,11768,12475,1071,...,13191,16796,19078,7093,12496,13056,15385,15805,10924,2705,14193,7417,13384,14209,4042,4543,6940,9599,9883,10988,17569,14798,16496,5997,7997
1/3/2024,11209,7029,5004,15017,11373,5649,10104,4453,5127,15905,4220,5894,9248,7215,12061,16490,7202,15831,6051,3426,17104,10680,15923,13968,11398,...,2938,15872,7786,7823,1204,9260,10640,7807,13283,13470,6203,13871,9113,17214,19754,11046,9394,13688,3650,8035,13950,17189,8603,5933,10796
1/2/2024,2722,3999,17068,14176,19561,4700,3868,14364,15485,2078,12429,3961,3889,12772,6013,8994,17809,10604,18632,16629,19883,9485,7451,1934,13355,...,2664,11745,10804,12666,2991,16461,13452,2500,12874,10383,7376,4644,10435,14369,6711,4652,10092,18652,10583,11124,7545,3864,16046,4969,11741
